# ETL: BUSINESS INTELLIGENCE

In [1]:
import pandas as pd

# 1. Carga de Datos

In [2]:
nm      = pd.read_csv("Datasets/name_basics.tsv", sep='\t')         # name_basics
tt      = pd.read_csv("Datasets/title_basics.tsv", sep='\t')        # title_basics
rating  = pd.read_csv("Datasets/title_ratings.tsv", sep='\t')       # title_ratings
tp      = pd.read_csv("Datasets/title_principals.tsv", sep='\t')    # title_principals
rev     = pd.read_csv("Datasets/revenue.csv")

C:\Users\alama\AppData\Local\Temp\ipykernel_18040\1471302542.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tt      = pd.read_csv("Datasets/title_basics.tsv", sep='\t')        # title_basics


In [5]:
# nm
# tt
# rating
rev
# tp

,tconst,budget,domestic,international,worldwide
0,tt1568911,66000000,79884879,97700000,177584879
1,tt0241760,13000000,29760152,1407236,31167388
2,tt5884052,150000000,144174568,305588070,449762638
3,tt0795438,5000000,10572742,154167,10726909
4,tt0462538,75000000,183135014,353279279,536414293
...,...,...,...,...,...
931892,tt23056928,-1,-1,-1,-1
931893,tt2138947,-1,-1,-1,-1
931894,tt0380075,-1,-1,-1,-1
931895,tt0887984,-1,-1,-1,-1


# 2. Limpieza de Datos

### 2.1. Estandarizacion

In [8]:
nm.replace({"\\N": None, "NaN": None}, inplace=True)
tt.replace({"\\N": None, "NaN": None}, inplace=True)
rating.replace({"\\N": None, "NaN": None}, inplace=True)
tp.replace({"\\N": None, "NaN": None}, inplace=True)
rev.replace({-1: None}, inplace=True)

### 2.2. Eliminacion de Datos

In [4]:
# name titles
nm = nm[nm["primaryName"].notnull()] # & nm["birthYear"].notnull()

In [5]:
tt = tt[tt["titleType"].isin(["movie", "tvMiniSeries", "tvMovie", "tvSeries"])]
tt.loc[tt['titleType'].isin(['movie', 'tvMovie']), 'endYear'] = tt.loc[tt['titleType'].isin(['movie', 'tvMovie']), 'startYear']
tt = pd.merge(tt, rating, on="tconst", how="left")
tt = pd.merge(tt, rev, on="tconst", how="left")

In [19]:
professions = nm[["nconst", "primaryProfession"]]
genres = professions.dropna(subset="primaryProfession")
professions = professions.assign(primaryProfession=professions["primaryProfession"].str.split(',')).explode("primaryProfession")
nm = nm.drop(columns=["primaryProfession", "knownForTitles"])

In [14]:
genres = tt[["tconst", "genres"]]
genres = genres.dropna(subset="genres")
genres = genres.assign(genres=genres["genres"].str.split(',')).explode("genres")
tt = tt.drop(columns=["genres", "isAdult"])

In [22]:
tp = tp.drop(columns=["job", "characters"])
tp = pd.merge(tp, tt["tconst"], on="tconst")
tp = pd.merge(tp, nm["nconst"], on="nconst")

,tconst,ordering,nconst,category,job,characters
0,tt0000009,1,nm0063086,actress,None,"[""Miss Geraldine Holbrook (Miss Jerry)""]"
1,tt0000009,2,nm0183823,actor,None,"[""Mr. Hamilton""]"
2,tt0006014,1,nm0183823,actor,None,"[""Henry Everard""]"
3,tt0006873,1,nm0183823,actor,None,"[""Robert Lovell""]"
4,tt0007129,2,nm0183823,actor,None,"[""Tom Silverton""]"
...,...,...,...,...,...,...
7157510,tt9916730,7,nm10538613,producer,associate producer,None
7157511,tt9916730,9,nm9785908,editor,None,None
7157512,tt9916754,2,nm3699941,self,None,"[""Self""]"
7157513,tt9916754,3,nm9813366,self,None,"[""Self""]"


,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,None,"[""Self""]"
1,tt0000001,2,nm0005690,director,None,None
2,tt0000001,3,nm0374658,cinematographer,director of photography,None
3,tt0000002,1,nm0721526,director,None,None
4,tt0000002,2,nm1335271,composer,None,None
...,...,...,...,...,...,...
55393270,tt9916880,4,nm10535738,actress,None,"[""Horrid Henry""]"
55393271,tt9916880,5,nm0996406,director,principal director,None
55393272,tt9916880,6,nm1482639,writer,None,None
55393273,tt9916880,7,nm2586970,writer,books,None
